In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import time

# Replace with your Azure Custom Vision keys and endpoint
training_endpoint = "<Your Custom Vision Training Endpoint>"
prediction_endpoint = "<Your Custom Vision Prediction Endpoint>"
training_key = "<Your Custom Vision Training Key>"
prediction_key = "<Your Custom Vision Prediction Key>"
project_id = "<Your Project ID>"
publish_iteration_name = "classifyModel"

# Authenticate
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(training_endpoint, credentials)

# Create a new project
project = trainer.create_project("My Classification Project")

# Add tags (categories)
tag1 = trainer.create_tag(project.id, "Category1")
tag2 = trainer.create_tag(project.id, "Category2")

# Upload images and tag them
base_image_location = "Screenshot 2023-11-08 233854.png"
for image_num in range(1, 11):
    with open(base_image_location + "Category1/image" + str(image_num) + ".jpg", "rb") as image_contents:
        trainer.create_images_from_data(project.id, image_contents.read(), [tag1.id])
for image_num in range(1, 11):
    with open(base_image_location + "Category2/image" + str(image_num) + ".jpg", "rb") as image_contents:
        trainer.create_images_from_data(project.id, image_contents.read(), [tag2.id])

# Train the model
iteration = trainer.train_project(project.id)
while iteration.status != "Completed":
    iteration = trainer.get_iteration(project.id, iteration.id)
    time.sleep(1)

# Publish the iteration
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id="<Your Prediction Resource ID>")

print("Model trained and published!")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_color_histogram(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Split the image into its respective channels (R, G, B)
    channels = cv2.split(image_rgb)
    colors = ("r", "g", "b")
    channel_names = ('Red', 'Green', 'Blue')
    
    # Plot the histograms
    plt.figure(figsize=(10, 5))
    plt.title("Color Histogram")
    plt.xlabel("Bins")
    plt.ylabel("Number of Pixels")
    
    for (channel, color, name) in zip(channels, colors, channel_names):
        # Calculate the histogram for the channel
        hist = cv2.calcHist([channel], [0], None, [256], [0, 256])
        plt.plot(hist, color=color)
        plt.xlim([0, 256])
        plt.legend(channel_names)
    
    plt.show()

# Path to the image file
image_path = 'Screenshot 2023-11-08 233854.png'

# Plot the color histogram for the image
plot_color_histogram(image_path)
